# Creating and Visualizing DEMs from LIDAR points

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook we will:
* Create high-quality DEMs from LiDAR point clouds and compute topographic parameters
* Create webmap visualizations


***

## 1. Import Python Packages and Start GRASS GIS Session

Import Python standard library and IPython packages we need. Start GRASS session in Nags Head project.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

gj.init("./nags_head/PERMANENT");

In [ ]:
!g.region -p

**Try it yourself!**

_Did you modify the computational region at the end of the previous notebook? If so, note if those changes are reflected in this notebook. Use `g.region` to switch back to the `jockeys_ridge` saved region._

<details>
    <summary>👉 <b>click for solution</b></summary>
    
The computational region is saved in the mapset so moving to a new notebook or starting a new session will not affect it.
```
!g.region -p region=jockeys_ridge
```
</details>

***

## 2. Create a DEM 

In [ ]:
lidar_files = sorted(Path('/data/grass-workshop').glob('*.las'))
lidar_files

### Mask low density areas

Interpolating a DEM from LiDAR points is only meaningful where there are adequate point coverage. So, our first step will be to mask areas that have low point densities before we interpolate. This also will make our interpolation run faster!

In [ ]:
!g.region region="jockeys_ridge" res=5

In [ ]:
gs.run_command("r.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014_count", method="n", flags="w")
gs.mapcalc(exp="JR_2014_mask=if(JR_2014_count == 0, null(), 1 )")

In [ ]:
mask = gj.Map(use_region=True)
mask.d_rgb(red="naip_2014.1", green="naip_2014.2", blue="naip_2014.3")
mask.d_rast(map="JR_2014_mask")
mask.show()

In [ ]:
!g.region res=0.5 # why do we change the resolution here?
!r.mask raster=JR_2014_mask

### Create DEM with Binning

In [ ]:
# Binning
!g.region res=2

In [ ]:
gs.run_command("r.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014", method="mean", flags="w")

In [ ]:
!r.mask -r

For you can also compute DEMs with a higher resolution than the point sampling distance using splines with either the [v.surf.rst](https://grass.osgeo.org/grass83/manuals/v.surf.rst.html) or [v.surf.bspline](https://grass.osgeo.org/grass83/manuals/v.surf.bspline.html) tools.

See Section 2.3.3 of [Hardin et al (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_2#Sec3) for more.

### Visualize DEM

_A static map made with_ `gj.Map`.

In [ ]:
elev = gj.Map()
elev.d_rast(map="JR_2014")
elev.show()

_A leaflet map made with_ `gj.InteractiveMap()`

In [ ]:
fig = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
fig.add_raster("JR_2014")
fig.add_layer_control()
fig.show()

### Create DEM with Splining

In [ ]:
!g.region -p res=2 grow=-200

In [ ]:
gs.run_command("v.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014", flags="wrc")

In [ ]:
# Interpolate using RST
gs.run_command("v.surf.rst", input="JR_2014", elev="JR_2014_spline", tension=40, npmin=100, segmax=15, dmin=2, mask="JR_2014_mask")

In [ ]:
!r.colors map=JR_2014 color=elevation

fig = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
fig.add_raster("JR_2014_spline")
fig.add_raster("JR_2014")
fig.add_layer_control()
fig.show()

**Try it yourself!**

`v.surf.rst` _is a power tool for interpolating LiDAR surfaces [(Mitasova et al, 2005)](https://ieeexplore.ieee.org/document/1522204). Look at the documentation for v.surf.rst and adjust the parameters to see how they affect the DEM. What happens if you change the resolution of the computational region?_

<details>
    <summary>👉 <b>click to see an example</b></summary>

```
! v.surf.rst --help
``` 
    
```python
gs.run_command("v.surf.rst", input="JR_2014", elev="JR_2014_spline", tension=10, smooth=0)
```
</details>

## 3. Bulk DEM Creation with Python

We'll use binning for this since it's faster.

In [ ]:
print(lidar_files)

In [ ]:
for file in lidar_files:
    gs.run_command("g.region", region="jockeys_ridge", res=5)
    
    # Create density mask
    output_name = file.stem
    gs.run_command("r.in.pdal", input=file, output=output_name+'_count', method="n", flags="w")
    gs.mapcalc(exp=f"{output_name}_mask=if({output_name}_count == 0, null(), 1 )")
    
    # Set Mask
    gs.run_command("g.region", res=0.5) # why do we change the resolution here?
    gs.run_command("r.mask", raster=f"{output_name}_mask")

    # Binning
    gs.run_command("g.region", res=3) # set resolution
    gs.run_command("r.in.pdal", input=file, output=output_name, method="mean", flags="w") #create binned raster
    gs.run_command("r.mask", flags="r") # remove mask
    print(f"imported {file.stem}")